<a href="https://colab.research.google.com/github/zeekx/build_dl_framework/blob/dev/steps/step12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scalar & Vector

Input:

scalar -> scalar

vector -> vector

Like Pytorch.tensor does.

np.array(1) + np.array(2) = 3(tuple)

np.array([1, 11]) + np.array([2, 22]) = [3, 33](np.array)


Make the parameter can get var arguments, let put '*' before it.


In [ ]:
import numpy as np

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None


    def set_creator(self, func):
        self.creator = func


# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self):
        fs = [self.creator]
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        while fs:
            f = fs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                fs.append(x.creator)



In [ ]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
class Function:
    def __call__(self, *inputs): # input:[x0, x1, ...]@Variable -> [y0, y1, ...]@Variable
        self.inputs = inputs
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #unwrap
        if not isinstance(ys, tuple):
            ys = (ys,)

        outputs = [Variable(as_array(y)) for y in ys]
        self.outputs = outputs
        for output in outputs:
            output.set_creator(self)

        return self.outputs if len(self.outputs) > 1 else self.outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [ ]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [ ]:
class Add(Function):
    def forward(self, a, b):
        y = a + b
        return (y,)

In [ ]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

def add(x0, x1):
  return Add()(x0, x1)

In Step11
```
x0, x1 = Variable(np.array(1)), Variable(np.array(2))
ab = [x0, x1]
add = Add()
y = add(ab)
```
Now, we can use the better way: add(x0, x1)

In [ ]:
x0, x1 = Variable(np.array(1)), Variable(np.array(2))
add = Add()
y = add(x0, x1)
y, y.data, type(y)

In [ ]:
a, b = Variable(np.array([1, 11])),  Variable(np.array([2, 22]))
add = Add()
y = add(a, b)
y, y.data, type(y)

Give a default grad value(array(1.0) for the last variable

In [ ]:
a = Variable(np.array(0.6))
b = Variable(np.array(0.4))
c = add(a, b)
c.data, type(c)